### Import modules

In [17]:
import os as os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image

### Define file names and paths

In [18]:
# note: path names adjusted to Windows, Mac/Linux may require path conversions

# experiment folder path
# default:        os.path.join(os.path.expanduser("~"), "Desktop", f"latest_{date.today()}")
PATH_EXP_FOLDER = os.path.join(os.path.expanduser("~"), "Desktop", f"latest_{date.today()}")

# laser round number (if not sure, refer to round numbers on cleave maps)
# default:        0
LASER_ROUND_NUM = 0

In [ ]:
# parameters for file/folder naming conventions
# do not change unless manually assigning files

# multichannel image folder name
PARAMS_MCI = "image_multichannel"
# mask image folder name
PARAMS_MSK = "image_mask"
# laser image folder name
PARAMS_LSR = "image_laser"
# cleave map folder name
PARAMS_MAP = "image_cleave_map"
# planned coordinates file name
PARAMS_PLN = "coord_planned.csv"
# recorded coordinates file name
PARAMS_CRD = "coord_recorded.csv"
# global mask file name
PARAMS_GLB = "image_mask_global.png"
# scan area list file name
PARAMS_SCT = "coord_scan_center.csv"
# bit scheme file name
PARAMS_BIT = "config_bit_scheme.csv"
# temp laser mask file name
PARAMS_TMP = "image_mask_tmp.png"

### Define plotting functions

In [ ]:
def pyplot_create_region(
        x: float,       # center x coordinate                       float / int
        y: float,       # center y coordinate                       float / int
        w: float,       # size of FOV over x axis                   float / int
        h: float,       # size of FOV over y axis                   float / int
        c = 'b',        # color to be used to plot the center       str / chr
        e = 'b',        # color to be used to plot the border       str / chr
        f = 'left',     # alignment of the center i to x axis       str / chr
        v = 'top',      # alignment of the center i to y axis       str / chr
        i = "",         # value to be displayed at the center       (printable)
        j = "",         # image to be displayed at the center       (file path)
        a = 1,          # alpha value of all graphic elements       float / int
        g = 1,          # alpha value of all glyphic elements       float / int
        b = False,      # flip image on W-E direction if True       bool
        d = False,      # flip image on N-S direction if True       bool
        r = 0,          # counter-clockwise rotation of image       int
        t = 0,          # counter-clockwise rotation of texts       int
):
    """
    ### Store a rectangle with width = w and height = h at (x,y), marked with i.
    
    `x` : center x coordinate.
    `y` : center y coordinate.
    `w` : size of FOV over x axis.
    `h` : size of FOV over y axis.
    -----------------------------------------------------------------------------------------------
    #### Optional:
    `c` : color to be used to plot the center. Default = `'b'` *(blue)*.
    `e` : color to be used to plot the border. Default = `'b'` *(blue)*.
    `f` : alignment of the center i to x axis. Default = `'left'`.
    `v` : alignment of the center i to y axis. Default = `'top'`.
    `i` : value to be displayed at the center. Default = `""`.
    `j` : image to be displayed at the center. Default = `""`.
    `a` : alpha value of all graphic elements. Default = `1`.
    `g` : alpha value of all glyphic elements. Default = `1`.
    `b` : flip image on W-E direction if True. Default = `False`.
    `d` : flip image on N-S direction if True. Default = `False`.
    `r` : counter-clockwise rotation of image. Default = `0`.
    `t` : counter-clockwise rotation of texts. Default = `0`.
    """
    # declare two lists to store corner coordinates
    corner_x = []
    corner_y = []
    # bottom left (start)
    corner_x.append(x - 0.5*w)
    corner_y.append(y - 0.5*h)
    # top left
    corner_x.append(x - 0.5*w)
    corner_y.append(y + 0.5*h)
    # top right
    corner_x.append(x + 0.5*w)
    corner_y.append(y + 0.5*h)
    # bottom right
    corner_x.append(x + 0.5*w)
    corner_y.append(y - 0.5*h)
    # bottom left (finish)
    corner_x.append(x - 0.5*w)
    corner_y.append(y - 0.5*h)
    # plot a center dot
    plt.plot(x, y, 'o', color=c, alpha=a)
    # plot i as label
    plt.text(x, y, i, ha=f, va=v, alpha=g, rotation=t)
    # plot j as image and rectX - rectY as lines
    if j != "":
        # open image with PIL
        img = Image.open(j)
        # flip or rotate the image if needed
        if b is True:
            img = img.transpose(method=Image.Transpose.FLIP_LEFT_RIGHT)
        if d is True:
            img = img.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)
        if r != 0:
            img = img.rotate(r)
        # store img, stretch its dimension to fit the current FOV
        ax = plt.gca()
        ax.imshow(
            np.fliplr(np.flipud(img)),
            extent = (x+0.5*w, x-0.5*w, y+0.5*h, y-0.5*h),
            alpha = a
        )
        # invert the axes back as imshow will invert x and y axis
        ax.invert_xaxis()
        ax.invert_yaxis()
        # graph rectX - recty with linestyle ':'
        plt.plot(corner_x, corner_y, ':', color=e, alpha=a)
    else:
        # graph rectX - recty with linestyle '-'
        plt.plot(corner_x, corner_y, '-', color=e, alpha=a)


def preview_stitching(
        images: list,
        coords: list,
        width,
        height,
        rotation = 0,
        center_index = False,
        export_result = False,
        flip_top_bottom = False,
        flip_left_right = False
    ):
    """
    Function: construct pyplot preview for listed images and coordinates.
    """
    try:
        # determine round number
        round_num = int(os.path.basename(images[0])[0:4]) - 1000
        img_name = ""
        # create a new image to save stitched laser images
        if export_result:
            map_fld = os.path.join(os.path.dirname(os.path.dirname(images[0])), PARAMS_MAP)
            mask = Image.open(os.path.join(map_fld, f"Round {round_num}.png"))
            img = Image.new('I;16', mask.size, 'black')
            img_name = os.path.join(map_fld, f"Round {round_num}.tif")
        # initialize pyplot regions
        for i, coord in enumerate(coords):
            pyplot_create_region(
                coord[0],
                coord[1],
                width,
                height,
                i = i if center_index else os.path.basename(images[i]),
                j = images[i],
                a = 0.75,
                g = 1,
                r = rotation,
                d = flip_top_bottom,
                b = flip_left_right,
                f = 'center',
                v = 'center',
                t = 45
            )
            if export_result:
                # area_num = int(os.path.basename(images[i])[5:9]) - 1000
                tmp = Image.open(images[i]).resize([366, 366])
                tmp = tmp.crop((33, 33, 366-33, 366-33)).rotate(180)
                nesw = []
                for j in range(2, 6):
                    nesw.append(int(coords[i][j]))
                img.paste(tmp.resize([300, 300]), nesw)
                # if needed, save stitched image as a new file
                img.save(img_name)
    except (AttributeError, IndexError, FileNotFoundError) as e:
        print(f"Warning: error occured during stitching preview: {e}")


def read_xycoordinates(csv_file):
    """
    Function: read row 1 and 2 as x and y coordinates (ignore row 0)
    """
    # read .csv, save xy coordinates in list, and print
    csv = pd.read_csv(csv_file).values.tolist()
    coords = []
    for row in csv:
        # if needed, invert x (row[1]) or y (row[2]) axis here
        coords.append([row[1], row[2]])
    # create regions in matplotlib based on the coordinates
    return coords


def app_cmc_lsr(exp_folder, round_num = 0, l_endswith = '.tif', m_endswith = '.png'):
    """
    Function: preview stitching for laser images, overlaying them onto mask images.
    """
    try:
        # clear previous plots
        plt.clf()
        # show mask stitching result as background
        file_endswith = m_endswith
        file_location = os.path.join(exp_folder, PARAMS_MSK)
        images = []
        for file in os.listdir(file_location):
            if file[-len(file_endswith):] == file_endswith:
                images.append(os.path.join(file_location, file))
        # acquire multichannel coordinates
        file_location = os.path.join(exp_folder, PARAMS_PLN)    # or PARAMS_CRD
        coords = read_xycoordinates(file_location)
        # check list lengths, initiate preview
        if len(images) != len(coords):
            print(f"Warning: found {len(images)} images, {len(coords)} coordinate pairs.")
            return
        preview_stitching(images, coords, 366, 366, 180, center_index=True)
        # acquire round number and image paths, return if error is encountered
        try:
            num_round = int(round_num)
            file_endswith = l_endswith
            file_location = os.path.join(exp_folder, PARAMS_LSR)
            # list_location = os.path.join(self.frm_ctl.ent_pth.get(), PARAMS_SCT)
            # center = pd.read_csv(
            #     list_location, keep_default_na = False, usecols=[1,2,4,5,6,7]).values.tolist()
            # coords = []
            coords = pd.read_csv(
                os.path.join(exp_folder, PARAMS_MAP, f"Round {num_round}.csv"),
                keep_default_na = False, usecols = [1,2,4,5,6,7]
            ).values.tolist()
            images = []
            for file in os.listdir(file_location):
                if file[-len(file_endswith):] == file_endswith and file[0:5] != "Round":
                    file_prefix_num = int(file[0:4]) - 1000
                    # file_affix_num = int(file[5:9]) - 1000
                    if file_prefix_num == num_round:
                        # coords.append(center[file_affix_num])
                        images.append(os.path.join(file_location, file))
            if len(images) == 0:
                print(f"Warning: found no laser images with matching round number {num_round}.")
                return
        except (ValueError, TypeError) as e:
            print(f"Warning: error accessing image paths/coordinates due to user input: {e}.")
            return
        # check list lengths, initiate preview
        if len(images) != len(coords):
            print(f"Warning: found {len(images)} images, {len(coords)} coordinate pairs.")
            return
        preview_stitching(images, coords, 366, 366, 180, export_result=True)
        # show plot preview window
        plt.gca().set_aspect('equal')
        plt.gcf().set_figheight(10)
        plt.gcf().set_figwidth(10)
        plt.show()
    except (AttributeError, IndexError, FileNotFoundError) as e:
        print(f"Warning: error occured in function `{app_cmc_lsr}`: {e}.")

### Read images and coordinates, show pyplot preview

In [ ]:
# in the designated experiment folder, preview a given round of laser-fluidic cycle
app_cmc_lsr(PATH_EXP_FOLDER, LASER_ROUND_NUM)